In [123]:
import os
import torch
import numpy as np
from diffusers import StableDiffusionPipeline, AutoencoderKL, UNet2DConditionModel
from diffusers.schedulers import PNDMScheduler
from transformers import CLIPTokenizer, CLIPTextModel
from typing import List, Tuple, Optional

HF_HOME = '../.hf_home'
os.environ['HF_HOME'] = HF_HOME

torch.manual_seed(42)
np.random.seed(42)

device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [95]:
# model_id = "CompVis/stable-diffusion-v1-4"
# pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, cache_dir=os.environ['HF_HOME'])
# pipe = pipe.to(device)
#
# vae = pipe.vae
# tokenizer = pipe.tokenizer
# text_encoder = pipe.text_encoder
# unet = pipe.unet
# scheduler = pipe.scheduler

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
class CustomDiffusion:
    def __init__(self, model_id: str = "CompVis/stable-diffusion-v1-4"):
        self.LATENT_SCALE_FACTOR=8
        self.BATCH_SIZE=1
        self.HEIGHT=512
        self.WIDTH=512
        self.model_id: str = model_id

        self.pipe: Optional[StableDiffusionPipeline] = None
        self.vae: Optional[AutoencoderKL] = None
        self.tokenizer: Optional[CLIPTokenizer] = None
        self.text_encoder: Optional[CLIPTextModel] = None
        self.unet: Optional[UNet2DConditionModel] = None
        self.scheduler: Optional[PNDMScheduler] = None

        self.__load_model()

    def __load_model(self):
        self.pipe = StableDiffusionPipeline.from_pretrained(self.model_id, torch_dtype=torch.float16, cache_dir=os.environ['HF_HOME'])
        self.vae = self.pipe.vae
        self.tokenizer = self.pipe.tokenizer
        self.text_encoder = self.pipe.text_encoder
        self.unet = self.pipe.text_encoder
        self.scheduler = self.pipe.scheduler

    def __encode_text(self, txt: str, dev=device) -> torch.Tensor:
        with torch.no_grad():
            tokenized = self.tokenizer(txt, return_tensors='pt', padding="max_length", max_length=tokenizer.model_max_length,
                                  truncation=True).input_ids.to(device)
            result = text_encoder(tokenized)[0].to(device)
            del tokenized
        return result

    def generate_text_embeddings(self, txt: str, dev=device) -> torch.Tensor:
        return torch.cat([self.__encode_text("", dev=dev), self.__encode_text(txt, dev=dev)])

    def prepare_diffusion(self, seed=42):
        latents = torch.randn(
            (
                self.BATCH_SIZE,
                unet.config.in_channels,
                self.HEIGHT // self.LATENT_SCALE_FACTOR,
                WIDTH // self.LATENT_SCALE_FACTOR,
            ),
            generator=generator,
            device=device,
            dtype=torch.float16,
        )


In [112]:
latent_scale_factor=8
batch_size=1
hEIGHT=512
WIDTH=512





In [110]:
encode_text(prompt).shape

torch.Size([1, 77, 768])

In [111]:
encode_text("").shape

torch.Size([1, 77, 768])

In [115]:
text_embeddings = generate_text_embeddings(prompt)
text_embeddings.shape

torch.Size([2, 77, 768])

In [ ]:
generator = torch.Generator("cuda").manual_seed(50)
image = pipe(prompt, generator=generator).images[0]
image